In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Loading the dataset shadow.p
import pickle
import torch

DATA_PATH = "/content/drive/MyDrive/AMLM/pickle/tinyimagenet/resnet34/shadow.p"
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=64, shuffle=False, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)

In [ ]:
print(type(dataset), len(dataset))

<class 'list'> 50000


In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    shadow_dataset = pickle.load(f)

In [ ]:
#Custom Dataset for the shadow model
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

train_data, test_data = train_test_split(shadow_dataset, test_size=0.2, random_state=42)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_dataset = CustomDataset(train_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=80, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=80, shuffle=False, num_workers=2)
print("Length of train_data:", len(train_data))
print("Length of test_data:", len(test_data))

Length of train_data: 40000
Length of test_data: 10000


In [ ]:
from torchvision import models

In [ ]:
num_classes = len(set([label for _, label in shadow_dataset]))
resnet34 = models.resnet34(pretrained=True)
resnet34.fc = nn.Linear(resnet34.fc.in_features, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 162MB/s]


In [ ]:
resnet34.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet34.parameters(), lr=0.0001)

In [ ]:
# Training the shadow model
num_epochs = 18

for epoch in range(num_epochs):
    resnet34.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet34(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

KeyboardInterrupt: 

In [ ]:
# Evaluating the shadow model
resnet34.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet34(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 62.17%


In [ ]:
print(num_classes)

200


In [ ]:
# Save the model
torch.save(resnet34, '/content/drive/MyDrive/AMLM/pickle/shadow_model.pth')

In [ ]:
shadow_model= torch.load("/content/drive/MyDrive/AMLM/pickle/shadow_model.pth", map_location=torch.device('cpu'))

In [ ]:
# Evaluating the shadow model
shadow_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet34(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 62.17%


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
shadow_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

**Generating the dataset for the attack model from the shadow model (member and non-member)**

In [ ]:
import torch.nn.functional as F

attack_model_dataset = []

# Iterate over the test dataset (non-members)
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet34(images)
        probabilities = F.softmax(outputs, dim=1)
        for i in range(len(images)):
            attack_model_dataset.append(probabilities[i].tolist() + [0])  # 0 for non-member

In [ ]:
print(attack_model_dataset[0])
print(len(attack_model_dataset))

[5.845411124028033e-07, 5.587342911894666e-06, 2.217735982412705e-06, 3.556857700459659e-05, 2.1925433202341083e-07, 4.969369342688879e-07, 7.181667882605325e-08, 1.5273000826709904e-05, 1.0379835657658987e-06, 5.557891995522368e-07, 2.8046534339409845e-07, 1.4812869153502106e-07, 1.583101294500011e-08, 1.1031273672301722e-08, 6.76423184131636e-08, 1.0715821190387942e-05, 0.00016664514259900898, 2.574406607891433e-07, 1.508455170551315e-06, 5.009425763091713e-08, 2.9444516158605438e-08, 9.138507834904885e-07, 2.1317507759022192e-08, 1.8596094264466956e-07, 8.054144018387888e-07, 2.4059903580564423e-07, 9.301848876930308e-06, 4.79447521684051e-07, 6.946162756094054e-08, 3.111842943326337e-07, 1.01738510238647e-06, 3.68215467005939e-08, 8.51351899200381e-07, 2.500266873539658e-07, 9.376564236163176e-08, 3.518029600968475e-08, 0.9994786381721497, 3.2986140467983205e-06, 9.886385669233277e-06, 4.380259269964881e-06, 2.1378023973284144e-07, 4.321781943872338e-06, 1.0792584959062879e-07, 1.3

In [ ]:
# Iterate over the train dataset (members)
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet34(images)
        probabilities = F.softmax(outputs, dim=1)
        for i in range(len(images)):
            attack_model_dataset.append(probabilities[i].tolist() + [1])  # 1 for member


In [ ]:
print(len(attack_model_dataset))

50000


In [ ]:
# Saving the dataset to a file
with open('/content/drive/MyDrive/AMLM/pickle/attack_model_dataset.pkl', 'wb') as f:
    pickle.dump(attack_model_dataset, f)

In [ ]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/AMLM/pickle/attack_model_dataset.pkl', 'rb') as f:
  attack_model_dataset = pickle.load(f)

**Attack Model**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Convert the attack model dataset to tensors
attack_model_dataset = torch.tensor(attack_model_dataset, dtype=torch.float32)
X = attack_model_dataset[:, :-1]
y = attack_model_dataset[:, -1]
print("Shape of X:", X.shape)
print("Shape of X:", y.shape)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train)
X_test_tensor = torch.tensor(X_test)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)  # Use long tensor for classification
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=80, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=80, shuffle=False)

Shape of X: torch.Size([50000, 200])
Shape of X: torch.Size([50000])


<ipython-input-8-09ad8192b606>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train)
<ipython-input-8-09ad8192b606>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test)
<ipython-input-8-09ad8192b606>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)  # Use long tensor for classification
<ipython-input-8-09ad8192b606>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().

In [ ]:
# Defining the attack model
class AttackModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttackModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x


In [ ]:
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
attack_model = AttackModel(input_size, hidden_size, output_size)

criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.001, weight_decay=1e-4)


In [ ]:
# Training the attack model
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
attack_model.to(device)

for epoch in range(num_epochs):
    attack_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = attack_model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.5291640939505677
Epoch 2, Loss: 0.4920820309419066
Epoch 3, Loss: 0.4899778845783782
Epoch 4, Loss: 0.48758026283897765
Epoch 5, Loss: 0.4839762712586416
Epoch 6, Loss: 0.47971525393664566
Epoch 7, Loss: 0.47458363797294495
Epoch 8, Loss: 0.469618632981222
Epoch 9, Loss: 0.4642420470986736
Epoch 10, Loss: 0.45885623814580645
Epoch 11, Loss: 0.4527015281866675
Epoch 12, Loss: 0.4460673395493259
Epoch 13, Loss: 0.4402377346879271
Epoch 14, Loss: 0.43619938358052135
Epoch 15, Loss: 0.43180057038999586
Epoch 16, Loss: 0.4279899174628192
Epoch 17, Loss: 0.4245422841751412
Epoch 18, Loss: 0.42151218686746134
Epoch 19, Loss: 0.41800048610527224
Epoch 20, Loss: 0.41553761184079463
Epoch 21, Loss: 0.41312997471796326
Epoch 22, Loss: 0.41116925114638186
Epoch 23, Loss: 0.4096364344908222
Epoch 24, Loss: 0.4081181907966801
Epoch 25, Loss: 0.40676968633175986
Epoch 26, Loss: 0.4054158771214964
Epoch 27, Loss: 0.40455924610705135
Epoch 28, Loss: 0.40333312824708684
Epoch 29, Loss: 

In [ ]:
# Evaluating the attack model
attack_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for tensors, labels in test_loader:
        tensors, labels = tensors.to(device), labels.to(device)
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
       # print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Test Accuracy: 84.21333333333334%


In [ ]:
# Saving the attack model
torch.save(attack_model.state_dict(), '/content/drive/MyDrive/AMLM/pickle/attack_model.pth')


In [ ]:
# attack_model= torch.load("/content/drive/MyDrive/AMLM/pickle/attack_model.pth", map_location=torch.device('cpu'))

In [ ]:
# from google.colab import files
# files.download('/content/drive/MyDrive/AMLM/pickle/attack_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Loading the target model and eval.p  Getting the Confidence Score and using it to evaluate the attack model**

In [ ]:
import torch
import torchvision.models as models

MODEL_PATH = '/content/drive/MyDrive/AMLM/models/resnet34_tinyimagenet.pth'
# Change the MODEL_PATH to your local model path

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_classes = 200  # Change num_classes to 200 for the Tiny ImageNet dataset
target_model = models.resnet34(num_classes=num_classes).to(device)

# Load the state dictionary
state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])

# Test accuracy
acc = state_dict['acc']
# Training epoch (start from 0)
epoch = state_dict['epoch']

print(f'Test Accuracy: {acc}')
print(f'Training Epoch: {epoch}')


Test Accuracy: 19.652
Training Epoch: 199


In [ ]:
import pickle
import torch

DATA_PATH = '/content/drive/MyDrive/AMLM/pickle/tinyimagenet/resnet34/eval.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    eval_dataset = pickle.load(f)

dataloader = torch.utils.data.DataLoader(
    eval_dataset, batch_size=64, shuffle=True, num_workers=2)

for batch_idx, (img, label, membership) in enumerate(dataloader):
    img = img.to(device)

In [ ]:
import pickle
import torch
import numpy as np
from PIL import Image

In [ ]:
confidence_scores = []
target_model.eval()
correct = 0
total = 0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for img, label, membership in dataloader:
        img = img.to(device)
        labels = label.to(device)
        outputs = target_model(img)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        for i in range(len(img)):
            confidence_scores.append(probabilities[i].tolist() + [membership[i]])
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = correct_predictions / total_predictions
    print("Accuracy:",accuracy)


Accuracy: 0.5531088082901554


In [ ]:
print(confidence_scores[0])

[4.20248179580085e-05, 2.202076393587049e-05, 1.3912255099057802e-06, 4.313547378842486e-06, 3.372127366674249e-06, 7.834866119083017e-05, 1.2336158761172555e-06, 1.5515395716647618e-05, 1.1035795068892185e-05, 1.92373354366282e-05, 1.6900634136618464e-06, 5.3807332733413205e-05, 1.4706936781294644e-05, 1.5372324924101122e-05, 9.998594578064512e-06, 2.3132149635785026e-06, 1.2362027518975083e-05, 1.5740157323307358e-05, 7.0876653808227275e-06, 2.846576080628438e-06, 4.78006768389605e-05, 7.577546057291329e-06, 7.1176928031491116e-06, 5.705901003238978e-06, 0.9961861968040466, 5.02266593684908e-05, 4.450123742572032e-05, 3.3763670217012987e-05, 4.8980506107909605e-05, 5.150815559318289e-05, 1.5150028048083186e-05, 9.869487257674336e-06, 8.727635758987162e-06, 2.0890556697850116e-05, 1.657484426687006e-05, 1.875170255516423e-06, 1.0862069757422432e-05, 2.3975351268745726e-06, 3.246859705541283e-05, 2.5392691895831376e-06, 3.4819418033293914e-06, 8.534350854461081e-06, 0.00021251279395073

In [ ]:
class CustomDatasetConf(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor = self.data[idx][:-1]
        label = self.data[idx][-1].item()
        return torch.tensor(tensor), label

In [ ]:
confidence_file_path = '/content/drive/MyDrive/AMLM/pickle/confidence_score.pkl'
with open(confidence_file_path, 'wb') as f:
    pickle.dump(confidence_scores, f)

In [ ]:
with open('/content/drive/MyDrive/AMLM/pickle/confidence_score.pkl', 'rb') as f:
  confidence_scores = pickle.load(f)

In [ ]:
print(confidence_scores[0])

[2.7119067453895696e-06, 8.747894071348128e-07, 5.7489796745358035e-06, 1.2092292536181048e-06, 2.6808553229784593e-05, 2.059331791315344e-06, 1.1050400644307956e-05, 4.6560166993003804e-06, 5.540515303437132e-06, 6.59618626741576e-06, 3.3318774512736127e-06, 4.678958248405252e-06, 9.169367558570229e-07, 6.952916464797454e-06, 2.056032099062577e-05, 8.630548222754442e-07, 2.9712862215092173e-06, 3.170624722770299e-06, 1.399035954818828e-05, 6.267458047659602e-06, 1.7104334801842924e-06, 1.615134533494711e-05, 2.163454837500467e-06, 7.141676178434864e-05, 7.760426115055452e-07, 9.826353561948054e-07, 8.277523875221959e-07, 4.85791906612576e-06, 9.64948071668914e-07, 1.0344783731852658e-06, 1.97544295588159e-06, 3.51494441019895e-06, 5.5340624385280535e-06, 5.843880444444949e-06, 3.5751418181462213e-06, 2.9997279852977954e-05, 1.2283820751690655e-06, 4.3064724195573945e-06, 2.071921471724636e-06, 5.1687970881175715e-06, 4.312587861932116e-06, 2.355925062147435e-06, 1.6651897567498963e-06

In [ ]:
conf_Score=CustomDatasetConf(confidence_scores)

In [ ]:
# Accessing the dataset
example_confidence_scores, example_membership = conf_Score[0]
print("Example confidence scores:", example_confidence_scores)
print("Example membership:", example_membership)

Example confidence scores: tensor([4.2025e-05, 2.2021e-05, 1.3912e-06, 4.3135e-06, 3.3721e-06, 7.8349e-05,
        1.2336e-06, 1.5515e-05, 1.1036e-05, 1.9237e-05, 1.6901e-06, 5.3807e-05,
        1.4707e-05, 1.5372e-05, 9.9986e-06, 2.3132e-06, 1.2362e-05, 1.5740e-05,
        7.0877e-06, 2.8466e-06, 4.7801e-05, 7.5775e-06, 7.1177e-06, 5.7059e-06,
        9.9619e-01, 5.0227e-05, 4.4501e-05, 3.3764e-05, 4.8981e-05, 5.1508e-05,
        1.5150e-05, 9.8695e-06, 8.7276e-06, 2.0891e-05, 1.6575e-05, 1.8752e-06,
        1.0862e-05, 2.3975e-06, 3.2469e-05, 2.5393e-06, 3.4819e-06, 8.5344e-06,
        2.1251e-04, 1.9539e-06, 3.0751e-06, 1.4463e-05, 1.5324e-05, 6.2817e-05,
        2.2666e-06, 9.4969e-06, 4.1931e-06, 1.9700e-05, 9.5376e-05, 6.7270e-06,
        4.5894e-06, 3.7776e-06, 2.8701e-05, 4.5245e-06, 3.2387e-05, 1.7031e-05,
        5.1241e-06, 1.4108e-06, 4.6627e-05, 3.4520e-06, 5.3858e-06, 1.9850e-06,
        6.2891e-06, 6.3219e-06, 6.8475e-05, 1.4745e-05, 2.3718e-06, 3.5590e-06,
        5.775

In [ ]:
print(len(conf_Score))

200


In [ ]:
# Creating a DataLoader for the confidence scores
confidence_scores_loader = DataLoader(conf_Score, batch_size=64, shuffle=False)

In [ ]:
print(len(confidence_scores))

200


In [ ]:
# Evaluating the attack model using the confidence score generated by the target model and eval.p
attack_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for tensors, label in confidence_scores_loader:
        tensors, label = tensors.to(device), label.to(device).float()
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
        total += tensors.size(0)
        correct += (predicted==label).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 93.5%


In [ ]:
# Evaluate the attack model
attack_model.eval()
final_results = []
correct = 0
total = 0

with torch.no_grad():
    for tensors, label in confidence_scores_loader:
        tensors, label = tensors.to(device), label.to(device).float()
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
        total += tensors.size(0)
        correct += (predicted == label).sum().item()
        final_results.extend(predicted.cpu().numpy().tolist())

print(f"Test Accuracy: {100 * correct / total}%")

print("First 10 predicted results:", final_results[:500])


Test Accuracy: 93.5%
First 10 predicted results: [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

**Result**

In [ ]:
predicted = np.array(final_results)

In [ ]:
print(predicted)

[1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.
 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1.
 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 0.]


In [ ]:
np.save('task2_resnet34_tinyimagenet.npy', predicted)

In [ ]:
from google.colab import files
files.download('task2_resnet34_tinyimagenet.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>